<a href="https://colab.research.google.com/github/allmore0/app_pprototipico/blob/main/PP_sesgo_min.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# -*- coding: utf-8 -*-
# Importando los módulos de Python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, recall_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
#import pymongo

file_name = "/content/Base Candidatos creada por IA (Gemini).csv"
df = pd.read_csv(file_name)

FileNotFoundError: [Errno 2] No such file or directory: '/content/Base Candidatos creada por IA (Gemini).csv'

In [ ]:
# Expectativas salariales (usando el punto medio) - Usado solo por CNN

df['Salario_Medio_MXN'] = pd.to_numeric(df['Sueldo_mensual'].astype(float))

In [ ]:
# MODELO CNN PARA PREDICCIÓN DE NSE (AMAI)

# Definición y mapeo de la Variable Objetivo (y): NSE (AMAI)
# Mapeo de categorías a enteros (de menor a mayor)
nse_mapping = {
    'E': 0, 'D': 1, 'Dmas': 2,
    'Cmenos': 3, 'C': 4, 'Cmas': 5,
    'B': 6, 'A': 7
}

# Columna original para el mapeo y columna renombrada para el BIAS
df['NSE_Mapped'] = df['Nivel_Socio_Económico(NSE_AMAI)'].map(nse_mapping).fillna(-1)
df = df[df['NSE_Mapped'] != -1]
unique_classes = sorted(df['NSE_Mapped'].unique())
class_to_idx = {cls: idx for idx, cls in enumerate(unique_classes)}
df['y'] = df['NSE_Mapped'].map(class_to_idx)

NUM_CLASSES = len(unique_classes)
candidate_target_names = [name for name, val in nse_mapping.items() if val in unique_classes]
print(f"Clases únicas de NSE (AMAI) encontradas: {candidate_target_names}")
print(f"Número de clases: {NUM_CLASSES}")

df.rename(columns={
    'Años de experiencia': 'Anios_de_experiencia',
    'Título_Principal': 'Titulo_Principal',
    'Certificación_1': 'Certificacion_1',
    'Certificación_2': 'Certificacion_2',
    'Estadística_Avanzada_Porcentaje': 'Estadistica_Avanzada_Pct',
    'Python_Porcentaje': 'Python_Pct',
    'R_Porcentaje': 'R_Pct',
    'SQL_Porcentaje': 'SQL_Pct',
    'Nivel_Socio_Económico(NSE_AMAI)': 'NSE_AMAI_Bias', # para el resumen de sesgos
    'Etnia_(Autodefinición)': 'Etnia_Autodefinicion'
}, inplace=True)

In [ ]:
numerical_features = [
    'Anios_de_experiencia', 'Edad', 'Python_Pct', 'R_Pct', 'SQL_Pct',
    'Estadistica_Avanzada_Pct', 'Salario_Medio_MXN'
]
categorical_features = ['Género'] # Usamos la técnica One-Hot Encoding para que se decifren numéricamente las categorías
df_encoded = pd.get_dummies(df, columns=categorical_features, drop_first=True)
X_cols = numerical_features + [col for col in df_encoded.columns if any(cat_col in col for cat_col in categorical_features) and col not in categorical_features]
# Valido columnas
X = df_encoded[X_cols].values
y = df_encoded['y'].values
NUM_FEATURES = X.shape[1]
print(f"Número de características finales: {NUM_FEATURES}")

# Final del preproceso para CNN
y_categorical = to_categorical(y, num_classes=NUM_CLASSES)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_reshaped = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)
X_train, X_test, y_train, y_test = train_test_split(
    X_reshaped, y_categorical, test_size=0.2, random_state=42)

In [ ]:
# Entrenamiento del modelo CNN
# Defino el modelo CNN (input_shape adaptado a NUM_FEATURES)
modelo = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=(NUM_FEATURES, 1)),
    Conv1D(64, kernel_size=3, activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(NUM_CLASSES, activation='softmax') # Ajustado a NUM_CLASSES
])

modelo.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Comeinza el entrenamiento
print("\nComenzando el entrenamiento del modelo...")
history = modelo.fit(X_train, y_train, epochs=80, batch_size=8,
                     validation_data=(X_test, y_test), verbose=0)
print("Entrenamiento finalizado.")

In [ ]:
# Evaluación del modelo CNN
plt.figure(figsize=(12, 4))
loss, acc = modelo.evaluate(X_test, y_test, verbose=0)
print(f"Evaluación del Modelo CNN")
print(f"Loss en Test: {loss:.4f} - Accuracy en Test: {acc:.4f}")

# Matriz de confusión para el NSE (AMAI)
pred = modelo.predict(X_test)
y_pred = np.argmax(pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Sensitivity (Recall) por clase
sensitivity = recall_score(y_true, y_pred, average=None, zero_division=0)
print("Sensibilidad (Recall) por Clase")
for i, s in enumerate(sensitivity):
    print(f"{candidate_target_names[i]}: {s:.2f}")

In [ ]:
# Predicción de un nuevo candidato
# Candidato de ejemplo con valores altos para probar la predicción
data_new = {col: 0 for col in X_cols}
data_new.update({
    'Anios_de_experiencia': 8.0,
    'Edad': 38,
    'Python_Pct': 0.98,  # Valores ya están en [0, 1]
    'R_Pct': 0.95,
    'SQL_Pct': 0.92,
    'Estadistica_Avanzada_Pct': 0.96,
    'Salario_Medio_MXN': 110000.0,
    # Categorical features for 'Masculino' and 'Prioritario'
    'Género_Masculino': 1,
    'Estatus_Prioritario': 1,
})
nuevo_candidato_df = pd.DataFrame([data_new], columns=X_cols).fillna(0)

# Procesamiento y Predicción
nuevo_candidato_scaled = scaler.transform(nuevo_candidato_df.values)
nuevo_candidato_reshaped = nuevo_candidato_scaled.reshape(1, NUM_FEATURES, 1)
pred_nuevo = modelo.predict(nuevo_candidato_reshaped, verbose=0)
clase_predicha_idx = np.argmax(pred_nuevo)

print("<<<<< Evaluación de Nuevo Candidato CNN >>>>>>")
print(f"Puntuación de la predicción (Softmax): {pred_nuevo[0]}")
print(f"Resultado: {candidate_target_names[clase_predicha_idx]}")

In [ ]:
# Evaluaciones de los resultados de candidatos (SCORING) con mitigación de sesgo
# Definiendo mappings y las funciones de puntuación
# A) Para los títulos principales
def score_titulo(titulo):
    titulo = str(titulo).lower()
    if 'ph.d.' in titulo or 'doctorado' in titulo or 'ia' in titulo:
        return 1.0
    elif 'maestría' in titulo or 'master' in titulo:
        return 0.8
    elif 'lic.' in titulo or 'ing.' in titulo or 'matemáticas' in titulo or 'computación' in titulo or 'ciencias de datos' in titulo:
        return 0.6
    else:
        return 0.3

# B) Certificaciones clave
def score_certificaciones(cert1, cert2):
    score = 0
    keywords = ['ml', 'ai', 'data', 'cloud', 'aws', 'azure', 'gcp', 'cert', 'specialty', 'recomendación']
    certs = [str(cert1).lower(), str(cert2).lower()]
    for cert in certs:
        if any(k in cert for k in keywords):
            score += 0.5 # 0.5 por certificación relevante, max 1.0
    return min(score, 1.0)

# C) Idiomas
level_map = {
    'a1': 0.1, 'a2': 0.2, 'b1': 0.4, 'b2': 0.6, 'c1': 0.8, 'c2': 1.0
}

def score_idiomas(nivel1, nivel2):
    score1 = level_map.get(str(nivel1).lower(), 0)
    score2 = level_map.get(str(nivel2).lower(), 0)
    # Sumar y normalizar a un resultado (score) máximo de 1.0
    return (score1 + score2) / 2.0

# Aplicamos funciones de puntuación al DataFrame
df['Score_Titulo'] = df['Titulo_Principal'].apply(score_titulo)
df['Score_Certificaciones'] = df.apply(lambda row: score_certificaciones(row['Certificacion_1'], row['Certificacion_2']), axis=1)
df['Score_Idiomas'] = df.apply(lambda row: score_idiomas(row['Nivel_idioma_1'], row['Nivel_idioma_2']), axis=1)

# 2. Cálculo del resultado (score) final ponderado
weights = {
    'Python_Pct': 0.25,
    'SQL_Pct': 0.20,
    'Estadistica_Avanzada_Pct': 0.15,
    'R_Pct': 0.05,
    'Score_Titulo': 0.15,
    'Score_Certificaciones': 0.10,
    'Score_Idiomas': 0.10
}

df['Score_Base'] = (
    df['Python_Pct'] * weights['Python_Pct'] +
    df['SQL_Pct'] * weights['SQL_Pct'] +
    df['Estadistica_Avanzada_Pct'] * weights['Estadistica_Avanzada_Pct'] +
    df['R_Pct'] * weights['R_Pct'] +
    df['Score_Titulo'] * weights['Score_Titulo'] +
    df['Score_Certificaciones'] * weights['Score_Certificaciones'] +
    df['Score_Idiomas'] * weights['Score_Idiomas']
)

# Ajuste del resultado (score) por los años de experiencia (mitigación de sesgo a la edad/años)
df['Experiencia_Multiplier'] = 1 + np.log1p(df['Anios_de_experiencia']) * 0.05
df['Score_Final'] = df['Score_Base'] * df['Experiencia_Multiplier']

# 3. Identificación del mejor candidato y Resumen de sesgos
best_candidate = df.loc[df['Score_Final'].idxmax()]

# Resumen de eliminación de sesgos
bias_cols = ['Edad', 'Género', 'Religión_ficticia', 'Afiliación_política_ficticia', 'NSE_AMAI_Bias', 'Etnia_Autodefinicion']
top_n = 10
top_candidates = df.sort_values(by='Score_Final', ascending=False).head(top_n)

print("\n\n*******************************************************")
print("\U0001F9D1\U0000200D\U0001F4BB\U0001F947 Candidato con Mejor Puntuación Final (Data Scientist Score):")
print("*******************************************************")
print(f"ID: {best_candidate['ID']}")
print(f"Nombre: {best_candidate['Nombre(s)']} {best_candidate['Apellido_Paterno']} {best_candidate['Apellido_Materno']}")
print(f"Años de Experiencia: {best_candidate['Anios_de_experiencia']}")
print(f"Score Final: {best_candidate['Score_Final']:.4f}")

print("\n\n*******************************************************")
print("\u2696\ufe0f Resumen de Eliminación de Sesgos (Comparación Top 10 vs. Población Total):")
print("*******************************************************")
for col in bias_cols:
    # Distribución en la población total
    total_dist = df[col].value_counts(normalize=True).mul(100).round(2)
    total_dist.name = 'Poblacion Total (%)'

    # Distribución en el top N
    top_dist = top_candidates[col].value_counts(normalize=True).mul(100).round(2)
    top_dist.name = f'Top {top_n} Seleccionados (%)'

    # Unión de las distribuciones y cálculo la 'Diferencia'
    summary = pd.concat([total_dist, top_dist], axis=1)
    summary.fillna(0, inplace=True)
    summary['Diferencia Absoluta (%)'] = (summary[f'Top {top_n} Seleccionados (%)'] - summary['Poblacion Total (%)']).round(2)

    print(f"\nColumna: {col}")
    print(summary.to_string())
    print("-" * 50)

# Guardar el DataFrame con los scores finales
output_df = df[['ID', 'Nombre(s)', 'Apellido_Paterno','Apellido_Materno', 'Anios_de_experiencia', 'Score_Final']].sort_values(by='Score_Final', ascending=False)
output_df.to_csv('candidatos_evaluados_score_final.csv', index=False)
print("\nEl archivo 'candidatos_evaluados_score_final.csv' ha sido generado con los resultados finales.")

# Task
- **Add MongoDB Import**: Add the `import pymongo` statement to the initial imports cell.
- **Load Web Form Data**: Load the data from the 'web_form_data.json' file into a temporary pandas DataFrame.
- **Append New Data**: Append the loaded web form data to the existing `df` DataFrame.
- **Final Task**: Confirm the MongoDB module import and the successful appending of web form data to the DataFrame.

## Add MongoDB Import

### Subtask:
Add the `import pymongo` statement to the initial imports cell.


**Reasoning**:
The subtask requires adding `import pymongo` to the initial imports cell. I will update the content of the first code cell to include this new import statement.



**Reasoning**:
The previous attempt to import `pymongo` failed because the module was not found. This indicates that the `pymongo` library needs to be installed in the environment. I will add a `!pip install pymongo` command in a new cell before the import statement.



In [ ]:
!pip install pymongo

**Reasoning**:
The `pymongo` library has been successfully installed in the previous step. Now, I will re-execute the initial imports cell, which now includes the `import pymongo` statement, to ensure all necessary modules are loaded without error.

